In [ ]:
########################## Montando um classificador para imputação de dados faltantes ##########################


####### Traduzir os dados em dados ordinários, em função da frequência deles (value_counts()) #######

state_encode_rev = df_students.State.value_counts().reset_index().reset_index()[["index","level_0"]].to_dict()['index']
city_encode_rev = df_students.City.value_counts().reset_index().reset_index()[["index","level_0"]].to_dict()['index']
university_encode_rev = df_students.UniversityId.value_counts().reset_index().reset_index()[["index","level_0"]].to_dict()['index']
course_encode_rev = df_students.CourseId.value_counts().reset_index().reset_index()[["index","level_0"]].to_dict()['index']

state_encode = dict((v,k) for k, v in state_encode_rev.items())
state_encode['NaN'] = np.nan
city_encode = dict((v,k) for k, v in city_encode_rev.items())
city_encode['NaN'] = np.nan
university_encode = dict((v,k) for k, v in university_encode_rev.items())
university_encode['NaN'] = np.nan
course_encode = dict((v,k) for k, v in course_encode_rev.items())
course_encode['NaN'] = np.nan


ordinal_state_cols_mapping = [{
    "col":"State",    
    "mapping": state_encode},
]
ordinal_city_cols_mapping = [{
    "col":"City",    
    "mapping": city_encode},
]
ordinal_university_cols_mapping = [{
    "col":"UniversityId",    
    "mapping": university_encode},
]
ordinal_course_cols_mapping = [{
    "col":"CourseId",    
    "mapping": course_encode},
]

state_encoder = OrdinalEncoder(mapping = ordinal_state_cols_mapping, 
                         return_df = True)  
city_encoder = OrdinalEncoder(mapping = ordinal_city_cols_mapping, 
                         return_df = True)  
university_encoder = OrdinalEncoder(mapping = ordinal_university_cols_mapping, 
                         return_df = True)  
course_encoder = OrdinalEncoder(mapping = ordinal_course_cols_mapping, 
                         return_df = True) 

df_students_enc = state_encoder.fit_transform(df_students)
df_students_enc = city_encoder.fit_transform(df_students_enc)
df_students_enc = university_encoder.fit_transform(df_students_enc)
df_students_enc = course_encoder.fit_transform(df_students_enc)

5- Imputation Using Multivariate Imputation by Chained Equation (MICE)
Main steps used in multiple imputations This type of imputation works by filling the missing data multiple times. Multiple Imputations (MIs) are much better than a single imputation as it measures the uncertainty of the missing values in a better way. The chained equations approach is also very flexible and can handle different variables of different data types (ie., continuous or binary) as well as complexities such as bounds or survey skip patterns. For more information on the algorithm mechanics, you can refer to the Research Paper

In [ ]:
####### Para treinar o classificador tipo 1 ########



train_enc = df_students_enc[["CourseId","UniversityId","State","City"]]
train_enc[train_enc == -1] = np.nan

# começa o treino #

imputed_training = mice(train_enc.values)
rounded_imputed_training = np.round(imputed_training)
pd_imputed_data = pd.DataFrame(rounded_imputed_training)

####### Terminou o treino sobre os dados e a imputação #######

### Renomeia as colunas do novo dataset ###

pd_imputed_data = pd_imputed_data.rename(columns = {0:"CourseId",1:"UniversityId",2:"State",3:"City"})
pd_imputed_data.head(10)

####### Decodificando de ordinários para nominais #######

pid_state = list(np.array(pd_imputed_data["State"]))
pid_city = list(np.array(pd_imputed_data["City"]))
pid_uni = list(np.array(pd_imputed_data["UniversityId"]))
pid_course = list(np.array(pd_imputed_data["CourseId"]))

temp_state = dict((str(key), value) for (key, value) in state_encode_rev.items())
temp_city = dict((str(key), value) for (key, value) in city_encode_rev.items())
temp_uni = dict((str(key), value) for (key, value) in uni_encode_rev.items())
temp_course = dict((str(key), value) for (key, value) in course_encode_rev.items())

str_pid_state = [str(x)[0] for x in pid_state]
str_pid_city = [str(x)[0] for x in pid_city]
str_pid_uni = [str(x)[0] for x in pid_uni]
str_pid_course = [str(x)[0] for x in pid_course]


for i in str_pid_state:
    str_pid_state[str_pid_state.index(i)] = temp_state[i]
for i in str_pid_city:
    str_pid_city[str_pid_city.index(i)] = temp_city[i]    
for i in str_pid_uni:
    str_pid_uni[str_pid_uni.index(i)] = temp_uni[i]    
for i in str_pid_course:
    str_pid_course[str_pid_course.index(i)] = temp_course[i]

In [ ]:
# daqui faz o processo reverso, dizer que nao da pra usar sklearn.preprocessing.OrdinalEncoder pq ele nao aceita
# NaN's
#temp = dict((np.float64(key), value) for (key, value) in state_encode_rev.items())
#sort_idx = np.argsort(temp.keys())
#idx = np.searchsorted(temp.keys(),pid,sorter = sort_idx)
#out = np.asarray(state_encode_rev.values())[sort_idx][idx]
#out

idx = np.nonzero(temp.keys() == pid[:,None])[1]
print(idx)
out = np.asarray(temp.values())[idx]

In [ ]:
# Normalizando com get_dummies()
df_students_dummies = pd.concat([pd.get_dummies(df_students['CourseId'],prefix='CourseId'),
                                 pd.get_dummies(df_students['UniversityId'],prefix='UniversityId'),
                                 pd.get_dummies(df_students['State'],prefix='State'),
                                 pd.get_dummies(df_students['City'],prefix='City')],
                                 axis = 1)



In [ ]:
# start the MICE training


imputed_training=mice(df_students_dummies.astype('float64'))
rounded_imputed_training = np.round(imputed_training)
pd.DataFrame(rounded_imputed_training)

In [ ]:
imputed_training=mice(train.values)
rounded_imputed_training = np.round(imputed_training)
pd.DataFrame(rounded_imputed_training)

In [ ]:
def translate_to_numeric():
    pass

def translate_to_original():
    pass